In [ ]:
import pandas as pd
from keras.layers import LSTM, Input, Bidirectional, TimeDistributed, Dropout, Dense, Activation
from keras.models import Model
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import numpy as np
import os

## Model

In [ ]:
def get_model(input_shape, output_shape, lr = 0.00001):
    inputs = Input(shape=input_shape)
    X = Dropout(0.5)(inputs)
    X = Dense(4096, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(2048, activation='relu')(X)
    X = Dense(2048, activation='relu')(X)
    X = Bidirectional(LSTM(1024, return_sequences=True))(X)
    outputs = TimeDistributed(Dense(output_shape, activation='softmax'))(X)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=tf.train.AdamOptimizer(lr),
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'],
    )
    
    return model

## Data

In [ ]:
def generator(path, mode, num_batches):
    counter = 0
    while True:
        if counter >= num_batches:
            counter = 0

        features = np.load(path + 'features_' + mode + '_' + str(counter) + '.npy')
        classes = np.load(path + 'classes_' + mode + '_' + str(counter) + '.npy')
        counter += 1
        yield features, classes

## Training

In [ ]:
model = get_model((2, 6144), 167)

In [ ]:
model.summary()

In [ ]:
model.fit_generator(
    generator('./features/lstm/2_steps/', 'train', 101),
    steps_per_epoch=101,
    epochs=50,
    validation_data=generator('./features/lstm/2_steps/', 'dev', 26),
    validation_steps=26,
)